In [ ]:
!nvidia-smi #to check gpu

Mon Apr 22 15:46:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##Installing and importing dependencies

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.8 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

In [ ]:
model_checkpoint='Helsinki-NLP/opus-mt-en-hi'

Model - https://huggingface.co/Helsinki-NLP/opus-mt-en-hi

Dataset used for fine tuning- https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [ ]:
raw_dataset=load_dataset('cfilt/iitb-english-hindi')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
raw_dataset #metadata

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
raw_dataset['train'][109]

{'translation': {'en': 'Validates application accessibility',
  'hi': 'अनुप्रयोग की पहुंचनीयता का सत्यापन करता है'}}

##Preprocessing


In [ ]:
#to create the embeddings
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer(["Hi, My name is Gayathri is Gayathri","This is my AIRIW Project"])

{'input_ids': [[10595, 2, 633, 300, 23, 36696, 10947, 3912, 23, 36696, 10947, 3912, 0], [239, 23, 155, 238, 13656, 389, 2431, 2170, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer(): #in the target language
  print(tokenizer(["अनुप्रयोग की पहुंचनीयता का सत्यापन करता है"]))

{'input_ids': [[2025, 15, 16155, 346, 24, 2014, 161, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length =128
max_target_length=128

source_lang="en"
target_lang="hi"

def preprocess_function(examples):
  inputs=[ex[source_lang] for ex in examples["translation"]]
  targets=[ex[target_lang] for ex in examples["translation"]]
  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [ ]:
preprocess_function(raw_dataset["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [ ]:
tokenized_datasets=raw_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
model=TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size =16
learning_rate=2e-5
weight_decay=0.01
num_train_epochs=10


In [ ]:
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf")


In [ ]:
generation_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf",pad_to_multiple_of=128)



In [ ]:
train_dataset=model.prepare_tf_dataset(
    tokenized_datasets["test"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
optimizer=AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 139s 375ms/step - loss: 3.7639 - val_loss: 3.9464


In [ ]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


##Testing

In [ ]:
input_text="How much did the GDP increase over the years? And then tell me what are your plans with your company"
tokenized=tokenizer([input_text],return_tensors='np')
out=model.generate(**tokenized,max_length=128)
print(out)

tf.Tensor(
[[61949  1774  6949     6   858   602 11900  6157    15  9944   331    22
      9   111   208  6168    25   143  6799     6   116  1281  4408    65
      5    40     0]], shape=(1, 27), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

कितने वर्षों के दौरान जीडिपी की वृद्धि हुई? और फिर मुझे बताओ कि अपनी कंपनी के साथ आपकी योजना क्या है।


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
!git config --global credential.helper store

In [ ]:
model.push_to_hub("EN-HI-translation")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("EN-HI-translation")

README.md:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nettem-Gayathri/EN-HI-translation/commit/f5fdf5f9dae9fb8813942e41523f6a5e8108f927', commit_message='Upload tokenizer', commit_description='', oid='f5fdf5f9dae9fb8813942e41523f6a5e8108f927', pr_url=None, pr_revision=None, pr_num=None)